## Preliminary preparations

In [1]:
from datetime import datetime
import os
from pathlib import Path

import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
pd.set_option('display.max_rows', 100)

In [2]:
logfile = "../logs/512G/2025-01-04-21-54-59-mainnet-beta-512G.log"
csv = "../logs/512G/tps_512gb.csv"

## Preparing Data Source

In [3]:
data = []

with open(logfile, 'r') as file:
    for line in file:
        if "datapoint: replay-slot-stats" in line:

            timestamp_match = re.search(r"\[(.*?)Z", line)
            transactions_match = re.search(r"total_transactions=(\d+)i", line)
            execute_us_match = re.search(r"execute_us=(\d+)i", line)
            
            if timestamp_match and transactions_match and execute_us_match:
                timestamp = timestamp_match.group(1)
                total_transactions_executed = int(transactions_match.group(1))
                execute_us = int(execute_us_match.group(1))

                data.append([timestamp, total_transactions_executed, execute_us])

df = pd.DataFrame(data, columns=['timestamp', 'total_transactions_executed', 'execute_us'])
df

,timestamp,total_transactions_executed,execute_us
0,2025-01-04T23:25:58.679585989,1872,657418
1,2025-01-04T23:25:59.035656422,1322,513342
2,2025-01-04T23:25:59.360325119,2811,547481
3,2025-01-04T23:25:59.710498540,1920,534702
4,2025-01-04T23:26:01.465814080,1963,485281
...,...,...,...
92149,2025-01-05T09:51:48.717028299,2215,389589
92150,2025-01-05T09:51:49.110718210,1736,416573
92151,2025-01-05T09:51:49.486106753,1782,401151
92152,2025-01-05T09:51:49.941188189,1400,388296


In [4]:
def convert_to_unixtime(timestamp):
    base_time, microseconds = timestamp.split('.')
    microseconds = microseconds[:6]
    dt = datetime.strptime(base_time, '%Y-%m-%dT%H:%M:%S')
    return int(dt.timestamp() * 1_000_000) + int(microseconds)

In [5]:
df['end'] = df['timestamp'].apply(convert_to_unixtime)
df['start'] = df['end'] - df['execute_us']
df['tps'] = df['total_transactions_executed'] * 1000000 / df['execute_us']
df

,timestamp,total_transactions_executed,execute_us,end,start,tps
0,2025-01-04T23:25:58.679585989,1872,657418,1736061958679585,1736061958022167,2847.503415
1,2025-01-04T23:25:59.035656422,1322,513342,1736061959035656,1736061958522314,2575.281197
2,2025-01-04T23:25:59.360325119,2811,547481,1736061959360325,1736061958812844,5134.424756
3,2025-01-04T23:25:59.710498540,1920,534702,1736061959710498,1736061959175796,3590.785148
4,2025-01-04T23:26:01.465814080,1963,485281,1736061961465814,1736061960980533,4045.079037
...,...,...,...,...,...,...
92149,2025-01-05T09:51:48.717028299,2215,389589,1736099508717028,1736099508327439,5685.478799
92150,2025-01-05T09:51:49.110718210,1736,416573,1736099509110718,1736099508694145,4167.336817
92151,2025-01-05T09:51:49.486106753,1782,401151,1736099509486106,1736099509084955,4442.217519
92152,2025-01-05T09:51:49.941188189,1400,388296,1736099509941188,1736099509552892,3605.496837


In [6]:
start_time = df.start.min() // 1000000 * 1000000
end_time = (df.start.max() + 1000000) // 1000000 * 1000000
time_step = 100000 # 10,000 microsends = .1 second

time_series = pd.DataFrame({
    't': np.arange(start_time, end_time + time_step, time_step)
})

def aggregate_tps(row, df):
    mask = (df['start'] <= row['t']) & (df['end'] >= row['t'])
    return df.loc[mask, 'tps'].sum()

time_series['sum_tps'] = time_series.apply(aggregate_tps, axis=1, df=df)
# window=10 means creating a moving average with the past 1 second's data. If you change it to 100, it would be the past 10 seconds
time_series['moving_average'] = time_series['sum_tps'].rolling(window=50, min_periods=1).mean()
time_series.to_csv(csv, index=False)


In [7]:
# Calculate the maximum TPS
max_tps = time_series['sum_tps'].max()

# Calculate the overall average TPS
average_tps = time_series['sum_tps'].mean()

# Print the results
print(f"Maximum TPS: {max_tps:,.2f}")
print(f"Average TPS: {average_tps:,.2f}")

# Convert the max TPS x-coordinate to the corresponding time
max_tps_time = time_series.loc[time_series['sum_tps'].idxmax(), 't']

# Calculate the total duration spent with 0 TPS
time_with_zero_tps = time_series[time_series['sum_tps'] == 0]
total_time_zero_tps = len(time_with_zero_tps) * time_step / 1_000_000  # Convert microseconds to seconds
total_time_zero_tps_hours = total_time_zero_tps / 3600  # Convert seconds to hours

print(f"Total time spent with 0 TPS: {total_time_zero_tps:.2f} seconds ({total_time_zero_tps_hours:.2f} hours)")

Maximum TPS: 119,564.91
Average TPS: 4,644.55
Total time spent with 0 TPS: 12259.70 seconds (3.41 hours)


In [8]:
# Display every 10 seconds
filtered_time_series = time_series[time_series['t'] % 10000000 == 0]
# from micro sec to sec
filtered_time_series['t'] = np.floor(filtered_time_series['t'] / 1000000)

filtered_time_series

,t,sum_tps,moving_average
20,1.736062e+09,0.000000,3675.155292
120,1.736062e+09,0.000000,5313.312373
220,1.736062e+09,0.000000,6112.045950
320,1.736062e+09,0.000000,7142.891286
420,1.736062e+09,0.000000,7125.327365
...,...,...,...
375120,1.736099e+09,0.000000,5724.038901
375220,1.736099e+09,4518.332367,4917.532657
375320,1.736099e+09,5776.587129,5145.987395
375420,1.736100e+09,7392.941833,5375.864550


In [9]:
# Calculate time elapsed in seconds from the start of the log
start_time = filtered_time_series['t'].min()  # First timestamp
filtered_time_series['elapsed_time'] = filtered_time_series['t'] - start_time
filtered_time_series['elapsed_time_hours'] = filtered_time_series['elapsed_time'] / 3600

# Convert elapsed time to HH:MM:SS format
filtered_time_series['elapsed_time_formatted'] = filtered_time_series['elapsed_time'].apply(
    lambda x: f"{int(x // 3600):02}:{int((x % 3600) // 60):02}:{int(x % 60):02}"
)

In [10]:
# Determine the range for the x-axis
x_min = filtered_time_series['elapsed_time_hours'].min() - 1  # One hour before the first data point
x_max = filtered_time_series['elapsed_time_hours'].max() + 1  # One hour after the last data point

fig = go.Figure()

fig.add_trace(go.Scatter(x=filtered_time_series['elapsed_time_hours'], y=filtered_time_series['sum_tps'], mode='markers', name='TPS', marker=dict(size=2)))
fig.add_trace(go.Scatter(x=filtered_time_series['elapsed_time_hours'], y=filtered_time_series['moving_average'], mode='lines', name='TPS Moving Average(1sec)', line=dict(width=0.5)))

# Add horizontal lines for max and average TPS
fig.add_hline(y=average_tps, line_dash="dot", line_color="green", annotation_text=f"Average TPS: {average_tps:.2f}", annotation_position="bottom right")
fig.add_hline(y=max_tps, line_dash="dash", line_color="red", annotation_text=f"Max TPS: {max_tps:.2f}", annotation_position="top right")

fig.update_layout(
    title=f"TPS Over Time | Total Time Spent with 0 TPS: {total_time_zero_tps:.2f} seconds ({total_time_zero_tps_hours:.2f} hours) | Max TPS: {max_tps:,.2f}",
    xaxis_title="Time Elapsed (hours)",
    yaxis_title="TPS",
    xaxis=dict(
        tickmode='linear',
        dtick=1,
        tickformat=".1f",
        range=[x_min, x_max]
    ),
    yaxis=dict(range=[0, 15000])
)


fig.show()


# ## Notes
# - If all data points are plotted on the graph, the points and lines become indistinguishable, so I plot them every 10 seconds.
# - TPS values above 15000 are excluded as outliers.
